## Здравствуйте, меня зовут *Имя Фамилия*, вот моё Задание №4 по курсу ПСАД-2020!

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt

from math import sqrt, factorial
from itertools import chain, permutations
from random import shuffle


import seaborn as sns
sns.set()

np.random.seed(228)

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

## Задача 1  

Задана обучающая выборка $\widetilde{S}$ размера $n \times 2$, включающая значения переменных $X$ и $Y$.

In [ ]:
from scipy.stats import linregress

def draw_linear(x, y, x_name='X', y_name='Y'):
    "Scatter x-y points and draw a LR line."
    slope, intercept, r, _, _ = linregress(x, y)
    x_max, x_min = x.max(), x.min()
    x_range = x_max - x_min
    x_lims = np.array([x_min - 0.1 * x_range, x_max + 0.1 * x_range])
    plt.xlabel(x_name)
    plt.ylabel(y_name)
    plt.plot(x, y, 'g+', label='Objects')
    plt.plot(x_lims, intercept + slope * x_lims, 'r--', label=r"LR line with $\rho \approx {:.3f}$".format(r))
    plt.legend()
    plt.show()

In [ ]:
S = np.load("data1.npy")
X, Y = S
draw_linear(X, Y)

* Оцените значимость коэффициента корреляции с помощью трёх тестов: критерия Стюдента, перестановочного теста и бутстреп-критерия. Под значимостью коэффициента в данном случае понимается его значимое отличие от нуля.

* Постройте графики нулевых распределений перестановочного и t-теста, а также распределение статистики бутстреп-теста. Попробуйте использовать функции `sns.lineplot, sns.distplot`. Для параметрического распределения носитель нужно определять самому, а для непараметрического можно рисовать гистограмму по данным как есть.

* Сравните полученные результаты и напишите выводы.

### Критерий Стьюдента

Проведите **двусторонний t-тест** для нулевой гипотезы о равенстве корреляции нулю. Для этого воспользуйтесь центральной предельной теоремой, приведя статистику к стандартизованной форме распределения Стьюдента . Полученное распределение $St(n - 2)$ будет **распределением нулевой гипотезы** $H_0$, а посчитанная статистика $T(X, Y)$ -- **его квантилью**. Достигаемый уровень значимости -- это вероятностная масса нулевого распределения, на которой суммарно получилось такое же или ещё более экстремальное значение статистики. В случае двустороннего теста заранее не известно, по какую сторону от нуля будет значение статистики, поэтому учитываются оба варианта.

In [ ]:
def student_significance(X, Y):
    # Your code goes here...
    return p_val

### Перестановочный тест

Проведите **двусторонний перестановочный тест** для нулевой гипотезы о равенстве корреляции нулю. Нулевое распределение порождается перебором $n!$ перестановок индексов одной из выборок. Достигаемый уровень значимости -- доля перестановок, на которых получилось такое же или ещё более экстремальное значение статистики. Перебор всех перестановок может быть очень большим, поэтому ограничиваются случайным подмножеством перестановок. В силу симметрии распределения, двусторонний тест учитывает абсолютное значение статистики. 

**Замечание**. Похоже, это очень непросто для создателей библиотек реализовать функцию перестановки, чтобы она возвращала _случайное подмножество_ перестановок. Функция `itertools.permutations` возвращает все возможные _упорядоченные_ перестановки, причём их невозможно перемешать или взять подвыборку, не раскрывая генератор. Функция `np.random.permutation` выполняет одну случайную перестановку, вот её как раз и можно записать в цикле.

In [ ]:
def permutation_test_significance(X, Y, n_permutations=100000):
    # Your code goes here...
    return perm_distr, p_val

### Бутстреп

Примените **двусторонний бутстреп-тест** о равенстве корреляции нулю. В бутстреп-тесте не строится распределение нулевой гипотезы, а вместо этого напрямую строится приближённое **распределение исследуемой статистики**. В оценке корреляции участвуют две выборки $X, Y$, поэтому для создания на их основе псевдовыборок методом Монте-Карло, мы сэмплируем с возвращением пары $(X_i, Y_i)$ и, считая корреляцию внутри каждой псевдовыборки, получаем бутстреп-распределение корреляции. 

Дальше по вычисленному бутстреп-распределению статистики нужно оценить **достигаемый уровень значимости (_p-value_)** критерия. Сначала построим два односторонних критерия. В них достигаемый уровень значимости -- это доля псевдовыборок, на которых корреляция меньше либо равно нулю (больше либо равна нулю). Дальше остаётся взять минимальное значение двух критериев (оно будет соответствовать тому, в какую сторону корреляция исходной выборки отклоняется от нуля) и умножить это значение на два (тем самым мы комбинаторно учитываем оба случая). Полученное значение и будет значением _p-value_ двустороннего бутстреп-критерия.

**Замечание**. Тоже относится к многомерному `np.random.choice` для случайного выбора пары точек. Но, похоже, что и его непросто реализовать, и у нас есть только одномерный вариант функции, который вполне успешно выполняется в цикле.

В последних версиях библиотеки добавили `np.random.Generator.choice` и `np.random.Generator.permutation`, в которых присутствует `axis`, позволяющий векторизовать многомерные операции. Если только вам будет интересно с этим разобраться, что, впрочем, вовсем необязательно, и главное -- это понимать математику методов.

In [ ]:
def bootstrap_test_significance(X, Y, n_pseudo_samples=10000):
    # Your code goes here...
    return boot_distr, p_val

Сравните результаты и сделайте выводы.

In [ ]:
print("Student significance: ", round(student_significance(X, Y), 5))
print("PT significance: ", round(permutation_test_significance(X, Y)[1], 5))
print("Boot significance: ", round(bootstrap_test_significance(X, Y)[1], 5))

**Выводы:**

## Задача 2

Сравните две группы $S_1$ и $S_2$ размера $10 \times n$ по переменным $X_1, \dots, X_{10}$.

In [ ]:
S1 = np.load("data2_1.npy")
S2 = np.load("data2_2.npy")

In [ ]:
X_from_S1, X_from_S2 = S1[0], S2[0] # Test it

### U-тест Манна-Уиттни-Уилкоксона

U-Тест проверяет нулевую гипотезы о том, что с равной вероятностью случайно выбранное значение из одной совокупности будет меньше или больше случайно выбранного значения из второй совокупности. 

Статистика U-теста эквивалентна значению ROC-AUC: (https://en.wikipedia.org/wiki/Mann–Whitney_U_test).

В данной реализации вы можете не учитывать корректировку связанных рангов, а также использовать нормальное приближение и вычислять достигаемый уровень значимости через центральную предельную теорему. Для вычисления статистики можно пользоваться функцией `sp.stats.rankdata`. Тест должен быть _двусторонним_.

In [ ]:
def mw_test(X_from_S1, X_from_S2):
    # Your code goes here...
    return p_val

### Критерий однородности Колмогорова-Смирнова
Существует две версии критерия Колмогорова-Смирнова: одновыборочный (_согласия_) и двухвыборочный (_однородности_).

https://ru.wikipedia.org/wiki/Критерий_согласия_Колмогорова

Критерий _согласия_ Колмогорова предназначен для проверки гипотезы о принадлежности выборки некоторому закону распределения, то есть проверки того, что эмпирическое распределение соответствует предполагаемой модели.

Критерий _однородности_ Смирнова используется для проверки гипотезы о принадлежности двух независимых выборок одному закону распределения, то есть о том, что два эмпирических распределения соответствуют одному и тому же закону.

**Замечание**. Написать векторный вариант этой функции может быть непросто, и вы можете посмотреть исходные коды библиотеки `scipy` и попробовать написать свою интерпретацию метода.

* https://github.com/scipy/scipy/blob/v1.4.1/scipy/stats/mstats_basic.py#L1235-L1282
* https://github.com/scipy/scipy/blob/v0.14.0/scipy/stats/stats.py#L3809
* https://github.com/scipy/scipy/blob/v1.4.1/scipy/stats/stats.py#L6087-L6282

In [ ]:
def ks_test(X_from_S1, X_from_S2):
    # Your code goes here...
    return prob

In [ ]:
res = defaultdict(list)

for i, (X_from_S1, X_from_S2) in enumerate(zip(S1, S2)):
    res["X"].append(i+1)
    res["Mann–Whitney"].append(mw_test(X_from_S1, X_from_S2))
    res["Kolmogorov–Smirnov"].append(ks_test(X_from_S1, X_from_S2))
    
pd.DataFrame(res)

Выясните, какие переменные являются значимыми на уровне $\alpha$ с учётом коррекции.

In [ ]:
alpha = 0.05

### Бонферрони


Простая коррекция _заданных_ уровней значимости: $ \alpha_1 = ... = \alpha_m = \alpha/m $. Альтернативный вид — переход к модифицированным _достигаемым_ уровням значимости: $\tilde{p}_i = \min(1, mp_i)$.

**Подсказка**. В программной реализации брать минимум не обязательно.

In [ ]:
def bonferroni_correction(p_vals, alpha=alpha):
    "Must return bool (!) array: reject/accept after correction"
    # Your code goes here...
    return is_rejected

### Бонферрони-Холма

_Нисходящий_ метод множественной проверки гипотез:

$$\alpha_1 = \dfrac{\alpha}{m}, \alpha_2 = \dfrac{\alpha}{m - 1}, \ldots, \alpha_i = \dfrac{\alpha}{m - i + 1}, \ldots, \alpha_m = \alpha.$$

Тогда из вариационного ряда, если $p_{i} \ge \alpha_i$, принять все нулевые гипотезы $H_{(i)}, H_{(i+1)} \ldots H_{(m)}$ и остановиться; иначе отвергнуть $H_{(i)}$ и продолжить. Каждый _достигаемый_ уровень значимости $p_{(i)}$ сравнивается со своим _заданным_ уровнем значимости $\alpha_i$. 

Иначе, модифицированные _достигаемые_ значимости (p-values): $\tilde{p}_{(i)} = \min \Big( 1, \max \big ( (m - i + 1)p_{(i)}, \tilde{p}_{(i - 1)}\big ) \Big )$ сравниваются с исходным _заданным_ уровнем значимости $\alpha$.

**Подсказка**. Воспользуйтесь функкцией `np.maximum.accumulate`. Перейти от вариационного ряда $\tilde{p}_{(i)}$ к исходному $\tilde{p}_i$ можно с помощью обратной перестановки сортированных индексов. Несложно построить обратную перестановку, используя тождество $f^{-1}f = e$ (https://stackoverflow.com/questions/11649577/how-to-invert-a-permutation-array-in-numpy?rq=1)

In [ ]:
def holm_correction(p_vals, alpha=alpha):
    "Must return bool (!) array: reject/accept after correction"
    # Your code goes here...
    return is_rejected

### Бенджамини-Хохберга

_Восходящий_ метод множественной проверки гипотез:

$$\alpha_1 = \dfrac{\alpha}{m}, \alpha_2 = \dfrac{2\alpha}{m}, \ldots, \alpha_i = \dfrac{i\alpha}{m}, \ldots, \alpha_m = \alpha.$$

Тогда из вариационного ряда, если $p_{i} \le \alpha_i$, отвергнуть все нулевые гипотезы $H_{(1)}, H_{(2)} \ldots H_{(i)}$ и остановиться; иначе принять $H_{(i)}$ и продолжить. Каждый _достигаемый_ уровень значимости $p_{(i)}$ сравнивается со своим _заданным_ уровнем значимости $\alpha_i$. 

Иначе, модифицированные _достигаемые_ значимости (p-values): $\tilde{p}_{(i)} = \min \big( 1,  \dfrac{mp_{(i)}}{i}, \tilde{p}_{(i + 1)} \big )$ сравниваются с исходным _заданным_ уровнем значимости $\alpha$.

**Подсказка**. `np.minimum.accumulate` и `[::-1]`

In [ ]:
def bendjamini_correction(p_vals, alpha=alpha):
    "Must return bool (!) array: reject/accept after correction"
    # Your code goes here...
    return is_rejected

Сравните результаты и сделайте выводы. Проверьте корректность вычислений сравнив значения с библиотечной функцией `statsmodels.stats.multitest.multipletests`.

*Дополнительная информация:* на самом деле лишь переменные $X_1$, $X_2$ и $X_3$ имеют одинаковое распределение.

Не забудьте сделать выводы!

**Выводы:**